In [ ]:
import pandas as pd
import boto3
import json

In [ ]:
"""
Load configuration parameters to AWS.
"""
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
config_dict = dict()

config_dict['KEY']                    = config.get('AWS','KEY')
config_dict['SECRET']                 = config.get('AWS','SECRET')

config_dict['CLUSTER_TYPE']       = config.get("DWH","CLUSTER_TYPE")
config_dict['NUM_NODES']          = config.get("DWH","NUM_NODES")
config_dict['NODE_TYPE']          = config.get("DWH","NODE_TYPE")
config_dict['IAM_ROLE_NAME']      = config.get("DWH", "IAM_ROLE_NAME")
config_dict['CLUSTER_IDENTIFIER'] = config.get("DWH","CLUSTER_IDENTIFIER")

config_dict['DB_NAME']                 = config.get("CLUSTER","DB_NAME")
config_dict['DB_USER']            = config.get("CLUSTER","DB_USER")
config_dict['DB_PASSWORD']        = config.get("CLUSTER","DB_PASSWORD")
config_dict['DB_PORT']               = config.get("CLUSTER","DB_PORT")

config_dict['POLICY1']               = config.get("IAM_ROLE", "POLICY1")
config_dict['POLICY2']               = config.get("IAM_ROLE", "POLICY2")
config_dict['ARN']               = config.get("IAM_ROLE", "ARN")

In [ ]:
"""
Create clients for EC2, S3, IAM and Redshift
"""
ec2 = boto3.resource('ec2',
                     region_name="us-west-2",
                     aws_access_key_id=config_dict['KEY'],
                     aws_secret_access_key=config_dict['SECRET']
                    )

s3 = boto3.resource('s3',
                     region_name="us-west-2",
                     aws_access_key_id=config_dict['KEY'],
                     aws_secret_access_key=config_dict['SECRET']
                   )

iam = boto3.client('iam',
                    aws_access_key_id=config_dict['KEY'],
                    aws_secret_access_key=config_dict['SECRET'],
                    region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                         region_name="us-west-2",
                         aws_access_key_id=config_dict['KEY'],
                         aws_secret_access_key=config_dict['SECRET']
                       )

print('Success creating clients.')

In [ ]:
"""
Create iam role.
"""    
try:
    print('Creating a new IAM Role')
    dwhRole = iam.create_role(Path='/',
                              RoleName=config_dict['IAM_ROLE_NAME'],
                              Description = "Allows Redshift clusters to call AWS services on your behalf.",
                              AssumeRolePolicyDocument=json.dumps(
                                  {'Statement': [{'Action': 'sts:AssumeRole',
                                                  'Effect': 'Allow',
                                                  'Principal': {'Service': 'redshift.amazonaws.com'}}],
                                   'Version': '2012-10-17'})
                              )    
except Exception as e:
    print(e)

try:
    print('Creating role policy')
    iam.attach_role_policy(RoleName=config_dict['IAM_ROLE_NAME'],
                           PolicyArn=config_dict['POLICY1']
                           )['ResponseMetadata']['HTTPStatusCode']
    iam.attach_role_policy(RoleName=config_dict['IAM_ROLE_NAME'],
                           PolicyArn=config_dict['POLICY2']
                           )['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)

roleArn = iam.get_role(RoleName=config_dict['IAM_ROLE_NAME'])['Role']['Arn']
print('role arn: ', roleArn)

In [ ]:
"""
Create cluster.
""" 
try:
    print('Creating redshift cluster')
    response = redshift.create_cluster(ClusterType=config_dict['CLUSTER_TYPE'],
                                       NodeType=config_dict['NODE_TYPE'],
                                       NumberOfNodes=int(config_dict['NUM_NODES']),
                                       DBName=config_dict['DB_NAME'],
                                       ClusterIdentifier=config_dict['CLUSTER_IDENTIFIER'],
                                       MasterUsername=config_dict['DB_USER'],
                                       MasterUserPassword=config_dict['DB_PASSWORD'],
                                       IamRoles=[roleArn] 
                                       )
except Exception as e:
    print(e)

In [ ]:
"""
Test if cluster is available.
"""
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=config_dict['CLUSTER_IDENTIFIER'])['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
"""
Open incoming TCP port.
""" 
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[-1]
    print(defaultSg)

    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(config_dict['DB_PORT']),
        ToPort=int(config_dict['DB_PORT'])
    )
except Exception as e:
    print(e)

In [ ]:
"""
Test connection.
"""
%load_ext sql
conn_string="postgresql://{}:{}@{}:{}/{}".format(config_dict['DB_USER'], config_dict['DB_PASSWORD'], myClusterProps['Endpoint']['Address'], config_dict['DB_PORT'],config_dict['DB_NAME'])
print(conn_string)
%sql $conn_string

In [ ]:
"""
Delete cluster.
"""
redshift.delete_cluster( ClusterIdentifier=config_dict['CLUSTER_IDENTIFIER'],  SkipFinalClusterSnapshot=True)


In [ ]:
"""
Test if cluster is deleted.
"""
myClusterProps = redshift.describe_clusters(ClusterIdentifier=config_dict['CLUSTER_IDENTIFIER'])['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
"""
Detach role policy, delete role.
"""
iam.detach_role_policy(RoleName=config_dict['IAM_ROLE_NAME'], PolicyArn=config_dict['POLICY1'])
iam.detach_role_policy(RoleName=config_dict['IAM_ROLE_NAME'], PolicyArn=config_dict['POLICY2'])
iam.delete_role(RoleName=config_dict['IAM_ROLE_NAME'])